Cell 0 PENJELASAN PROGRAM
# 00 — Extract Hand Landmarks (ASL Alphabet) → CSV

## Tujuan
Mengubah gambar dataset ASL Alphabet menjadi fitur numerik berbasis **21 hand landmarks** (x,y,z) dari MediaPipe.
Outputnya CSV:
- `data/landmarks/asl_alphabet_landmarks.csv`
  - kolom: `label` + `63 fitur` (21 titik × 3 koordinat)

## Kenapa landmarks?
Lebih ringan dari pixel dan cocok untuk KNN, karena kami memodelkan bentuk/pose tangan.

Cell 1 (Code) — Import & path dasar

In [6]:
# ====== IMPORT ======
from pathlib import Path          # path file/folder yang aman di Windows/Mac/Linux
import csv                        # menulis CSV
import random                     # shuffle/sampling
import cv2                         # baca gambar (OpenCV)
import numpy as np                # komputasi numerik
import mediapipe as mp            # MediaPipe Tasks API

# ====== PATH PROJECT (notebook ada di folder /notebooks) ======
ROOT = Path.cwd().parent          # naik 1 level: dari /notebooks ke root project
TASK_PATH = ROOT / "assets" / "hand_landmarker.task"                 # model MediaPipe (.task)
DATASET_ROOT = ROOT / "data" / "raw" / "asl-alphabet"                # dataset ASL Alphabet (hasil extract)
OUT_CSV = ROOT / "data" / "landmarks" / "asl_alphabet_landmarks.csv" # output CSV

# ====== PARAMETER ======
MAX_PER_CLASS = 300               # batasi gambar per kelas agar cepat (boleh dinaikkan untuk akurasi)
SEED = 42                         # seed agar shuffle konsisten
EXCLUDE = {"nothing", "space", "del", "delete"}  # opsional: kelas yang tidak dipakai

# ====== DEBUG PRINT ======
print("ROOT:", ROOT)                                  # cek root benar
print("TASK_PATH:", TASK_PATH, "exists:", TASK_PATH.exists())        # cek file model
print("DATASET_ROOT:", DATASET_ROOT, "exists:", DATASET_ROOT.exists())# cek dataset folder
print("OUT_CSV:", OUT_CSV)                            # lokasi output CSV


ROOT: d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks
TASK_PATH: d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\assets\hand_landmarker.task exists: True
DATASET_ROOT: d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet exists: True
OUT_CSV: d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\landmarks\asl_alphabet_landmarks.csv


Cell 2
## Kenapa sering “Total ditulis = 0”?
Biasanya karena:
1) Path dataset salah / dataset belum diekstrak.
2) Folder train dobel: `asl_alphabet_train/asl_alphabet_train/...`
3) Tidak ada gambar terdeteksi (jumlah gambar = 0).

Di cell berikut, kita auto-detect folder train yang benar dan cek jumlah gambar dulu.

Cell 3 (Code) — Auto-detect folder train + cek gambar

In [7]:
def find_train_dir(base: Path) -> Path:
    """
    Mencari folder train yang benar (berisi subfolder kelas A/B/C/.. dan file gambar).
    Mengatasi struktur dataset yang kadang dobel folder.
    """
    # kandidat struktur yang umum
    candidates = [
        base / "asl_alphabet_train",                         # struktur normal
        base / "asl_alphabet_train" / "asl_alphabet_train",  # struktur dobel
    ]

    # ekstensi gambar yang diterima
    exts = {".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"}

    # 1) coba kandidat cepat
    for c in candidates:
        if c.exists() and c.is_dir():                        # pastikan kandidat adalah folder
            subdirs = [d for d in c.iterdir() if d.is_dir()] # subfolder kelas
            if len(subdirs) >= 5:                            # minimal ada beberapa kelas
                # cek beberapa folder kelas, minimal ada 1 gambar
                for d in subdirs[:10]:
                    has_img = any(
                        (p.is_file() and p.suffix in exts)   # file gambar
                        for p in d.iterdir()
                    )
                    if has_img:
                        return c                              # kandidat ini benar

    # 2) fallback scan: cari folder yang punya subfolder "A"
    for c in base.rglob("*"):
        if c.is_dir() and (c / "A").is_dir():
            return c

    # kalau gagal total
    raise FileNotFoundError("Tidak ketemu folder train. Cek struktur data/raw/asl-alphabet/")

# ====== VALIDASI FILE/FOLDER ======
if not TASK_PATH.exists():
    raise FileNotFoundError(f"Model task tidak ditemukan: {TASK_PATH}")

if not DATASET_ROOT.exists():
    raise FileNotFoundError(f"Dataset root tidak ditemukan: {DATASET_ROOT}")

# tentukan folder train
TRAIN_DIR = find_train_dir(DATASET_ROOT)
print("✅ TRAIN_DIR:", TRAIN_DIR)

# hitung total gambar (untuk memastikan tidak 0)
all_imgs = (
    list(TRAIN_DIR.rglob("*.jpg")) +
    list(TRAIN_DIR.rglob("*.jpeg")) +
    list(TRAIN_DIR.rglob("*.png"))
)
print("🖼️ Total gambar terdeteksi:", len(all_imgs))

# kalau 0 berarti dataset belum siap
if len(all_imgs) == 0:
    raise RuntimeError("Gambar terdeteksi 0. Dataset belum diekstrak benar atau path salah.")

# cek beberapa contoh path gambar (debug)
print("Contoh 5 gambar pertama:")
for p in all_imgs[:5]:
    print(" -", p)


✅ TRAIN_DIR: d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet\asl_alphabet_train
🖼️ Total gambar terdeteksi: 87000
Contoh 5 gambar pertama:
 - d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet\asl_alphabet_train\A\A1.jpg
 - d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet\asl_alphabet_train\A\A10.jpg
 - d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet\asl_alphabet_train\A\A100.jpg
 - d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet\asl_alphabet_train\A\A1000.jpg
 - d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\raw\asl-alphabet\asl_alphabet_train\A\A1001.jpg


Cell 4
## Setup MediaPipe Hand Landmarker (IMAGE Mode)
Untuk dataset gambar statis pakai:
- `running_mode=IMAGE`
- panggil `landmarker.detect(mp_image)`

(Reatime webcam nanti pakai mode VIDEO dan `detect_for_video` + timestamp).


Cell 5 (Code) — Inisialisasi HandLandmarker IMAGE + fungsi fitur

In [8]:
# ====== ALIAS CLASS DARI MEDIAPIPE TASKS ======
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# ====== OPTIONS UNTUK IMAGE MODE ======
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=str(TASK_PATH)),  # path model .task
    running_mode=VisionRunningMode.IMAGE,                       # IMAGE untuk gambar statis
    num_hands=1,                                                # cukup 1 tangan
    min_hand_detection_confidence=0.5,                          # threshold deteksi
    min_hand_presence_confidence=0.5,                           # threshold presence
    min_tracking_confidence=0.5,                                # threshold tracking
)

def landmarks_to_features(lms) -> np.ndarray:
    """
    21 landmarks -> 63 fitur.
    Normalisasi agar lebih robust:
    - translasi: wrist (landmark 0) jadi origin
    - skala: dibagi skala jarak maksimum 2D
    """
    pts = np.array([[lm.x, lm.y, lm.z] for lm in lms], dtype=np.float32)  # (21,3)
    pts = pts - pts[0]                                                   # translasi-invariant
    scale = np.max(np.linalg.norm(pts[:, :2], axis=1))                   # skala (2D)
    if scale < 1e-6:                                                     # hindari pembagian nol
        scale = 1.0
    pts = pts / scale                                                    # scale-invariant
    return pts.reshape(-1)                                               # (63,)

# pastikan folder output ada
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)


Cell 5 (Code) — Inisialisasi HandLandmarker IMAGE + fungsi fitur

In [9]:
# ====== ALIAS CLASS DARI MEDIAPIPE TASKS ======
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# ====== OPTIONS UNTUK IMAGE MODE ======
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=str(TASK_PATH)),  # path model .task
    running_mode=VisionRunningMode.IMAGE,                       # IMAGE untuk gambar statis
    num_hands=1,                                                # cukup 1 tangan
    min_hand_detection_confidence=0.5,                          # threshold deteksi
    min_hand_presence_confidence=0.5,                           # threshold presence
    min_tracking_confidence=0.5,                                # threshold tracking
)

def landmarks_to_features(lms) -> np.ndarray:
    """
    21 landmarks -> 63 fitur.
    Normalisasi agar lebih robust:
    - translasi: wrist (landmark 0) jadi origin
    - skala: dibagi skala jarak maksimum 2D
    """
    pts = np.array([[lm.x, lm.y, lm.z] for lm in lms], dtype=np.float32)  # (21,3)
    pts = pts - pts[0]                                                   # translasi-invariant
    scale = np.max(np.linalg.norm(pts[:, :2], axis=1))                   # skala (2D)
    if scale < 1e-6:                                                     # hindari pembagian nol
        scale = 1.0
    pts = pts / scale                                                    # scale-invariant
    return pts.reshape(-1)                                               # (63,)

# pastikan folder output ada
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)


Cell 6
## Ekstraksi → CSV
Langkah inti:
1) Loop setiap folder kelas (A, B, C, ...)
2) Baca gambar (OpenCV = BGR)
3) Convert ke RGB (MediaPipe butuh RGB/sRGB)
4) `detect()` untuk dapat landmarks
5) Simpan fitur 63D + label ke CSV

Cell 7 (Code) — Loop kelas, deteksi, tulis CSV

In [10]:
# ====== TULIS HEADER CSV ======
with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["label"] + [f"f{i}" for i in range(63)])  # label + 63 fitur

# seed random
random.seed(SEED)

# ambil semua folder kelas
class_folders = [p for p in TRAIN_DIR.iterdir() if p.is_dir()]
print("Jumlah folder kelas terdeteksi:", len(class_folders))

if not class_folders:
    raise RuntimeError(f"Tidak ada folder kelas di {TRAIN_DIR}. Cek struktur dataset.")

total_written = 0    # total baris sukses ditulis
total_nohand = 0     # total gambar tanpa tangan terdeteksi

# ====== PROSES DETEKSI ======
with HandLandmarker.create_from_options(options) as landmarker:
    # buka CSV sekali saja (lebih cepat daripada open/close per baris)
    with open(OUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # loop per kelas
        for folder in sorted(class_folders, key=lambda p: p.name.lower()):
            label_lower = folder.name.lower()

            # skip kelas tertentu (opsional)
            if label_lower in EXCLUDE:
                continue

            # kumpulkan gambar kelas
            imgs = (
                list(folder.rglob("*.jpg")) +
                list(folder.rglob("*.jpeg")) +
                list(folder.rglob("*.png"))
            )

            # kalau kelas kosong, skip
            if not imgs:
                print(f"[SKIP] {folder.name} (tidak ada gambar)")
                continue

            # shuffle dan batasi
            random.shuffle(imgs)
            imgs = imgs[:MAX_PER_CLASS]

            wrote_this = 0  # sukses untuk kelas ini

            for img_path in imgs:
                bgr = cv2.imread(str(img_path))               # baca gambar (BGR)
                if bgr is None:
                    continue

                rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)    # ubah ke RGB untuk MediaPipe
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

                result = landmarker.detect(mp_image)          # IMAGE mode -> detect()

                # kalau tidak ada tangan terdeteksi, skip
                if not result.hand_landmarks:
                    total_nohand += 1
                    continue

                feat = landmarks_to_features(result.hand_landmarks[0])  # fitur (63,)

                # tulis: label + 63 fitur
                writer.writerow([folder.name.upper()] + feat.tolist())

                total_written += 1
                wrote_this += 1

            print(f"[{folder.name.upper()}] wrote={wrote_this}/{len(imgs)}")

print("\n✅ Selesai ekstraksi!")
print("CSV:", OUT_CSV)
print("Total ditulis:", total_written, "(syarat tugas >= 100)")
print("Skip(no hand):", total_nohand)


Jumlah folder kelas terdeteksi: 29
[A] wrote=220/300
[B] wrote=216/300
[C] wrote=190/300
[D] wrote=243/300
[E] wrote=219/300
[F] wrote=286/300
[G] wrote=244/300
[H] wrote=239/300
[I] wrote=231/300
[J] wrote=261/300
[K] wrote=266/300
[L] wrote=254/300
[M] wrote=161/300
[N] wrote=126/300
[O] wrote=217/300
[P] wrote=190/300
[Q] wrote=218/300
[R] wrote=248/300
[S] wrote=252/300
[T] wrote=239/300
[U] wrote=266/300
[V] wrote=265/300
[W] wrote=244/300
[X] wrote=225/300
[Y] wrote=268/300
[Z] wrote=235/300

✅ Selesai ekstraksi!
CSV: d:\KULIAH\SEMESTER 5\KECERDASAN BUATAN\TUBES 3\asl-knn-landmarks\data\landmarks\asl_alphabet_landmarks.csv
Total ditulis: 6023 (syarat tugas >= 100)
Skip(no hand): 1777
